# Making Predictions: An Example Using the Beer Dataset

We will go through the proccess for predicting Y quantities with SIMCA-Q.

We will use a SIMCA project where two OPLS models have been built using the BEER dataset typically used in SIMCA courses. The picture below shows the initial observations and variables of the original dataset used in this project:

![Original Dataset](Images/Dataset1.jpg)

<img src="Images/Dataset1.jpg">

<img src="https://github.com/OEM-Sartorius-Data-Analytics/SimcaQ_Python_Scripting_Guide/tree/main/06_PredictionInterface_1/Images/Dataset1.png">

<img src="Images/Dataset1.png" />